In [1]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

from preprocessing import limpiar_datos, aplicar_dummy_variables_encoding, eliminar_features, entrenar_iterative_imputer, imputar_missings_iterative, reduccion_PCA, normalizar_dataframe, entrenar_normalizador_standard, entrenar_normalizador_minmax
from graficos_modelos import mostrar_reporte_clasificacion, graficar_auc_roc,graficar_matriz_confusion
from funciones_auxiliares import traer_datasets, traer_dataset_prediccion_final, separar_dataset, encontrar_hiperparametros_RGSCV, mapear_target_binario

# K Nearest Neighbors

### Obtención de los datos

Traemos datasets y particionamos

In [2]:
df, df_sin_target, solo_target = traer_datasets()

X_train, X_test, y_train, y_test = separar_dataset(df_sin_target, solo_target)

X_train.is_copy=False
X_test.is_copy=False
y_train.is_copy=False
y_test.is_copy=False

y_train.set_index('id', inplace=True)
y_train = y_train.sort_values(by=['id'], ascending=True).copy()

y_test.set_index('id', inplace=True)
y_test = y_test.sort_values(by=['id'], ascending=True).copy()


### Definiendo distintos preprocesamientos

Definiremos entonces dos preprocesamientos distintos a comparar para este modelo, y quedarnos con el mejor de ellos cuando probemos en holdout.

In [3]:
def preprocesamiento_1(df_original:pd.DataFrame, imputer=None, normalizador=None):
    df = df_original.copy(deep=True)
    df = limpiar_datos(df)
    df = aplicar_dummy_variables_encoding(df, ['llovieron_hamburguesas_hoy'])
        
    eliminar_features(df, df.columns.difference(['id', 'horas_de_sol', 'humedad_tarde', 'nubosidad_tarde', 'llovieron_hamburguesas_hoy_si', 'llovieron_hamburguesas_hoy_nan']))
    
    if(imputer is None):
        imputer = entrenar_iterative_imputer(df)
    df = imputar_missings_iterative(df, imputer)
    
    if(normalizador is None):
        normalizador = entrenar_normalizador_standard(df)
    df = normalizar_dataframe(df, normalizador)
    
    return df, imputer, normalizador

Sabiendo que vamos a aplicar KNN, el cuál tiende a dejar de funcionar con dimensionalidad elevada, decidimos que el primer preprocesamiento realice una selección de features que se quede solamente con aquellas que determinamos como importantes en la primera parte del trabajo práctico. Adicionalmente además de realizar las conversiones e imputaciones necesarias, normalizaremos los datos con StandardScaler sabiendo que knn se beneficia de ello.

In [4]:
def preprocesamiento_2(df_original:pd.DataFrame, imputer=None, normalizador=None):
    df = df_original.copy(deep=True)
    df = limpiar_datos(df)
    df = aplicar_dummy_variables_encoding(df, ['llovieron_hamburguesas_hoy'])
    eliminar_features(df, ['dia','barrio', 'direccion_viento_tarde', 'direccion_viento_temprano', 'rafaga_viento_max_direccion'])
    
    if(imputer is None):
        imputer = entrenar_iterative_imputer(df)
    df = imputar_missings_iterative(df, imputer)
    
    if(normalizador is None):
        normalizador = entrenar_normalizador_minmax(df)
    df= normalizar_dataframe(df, normalizador)
    
    df = reduccion_PCA(df, 5)
    
    return df, imputer, normalizador

Para el segundo tipo de preprocesamiento, optaremos por eliminar menos features inicialmente (solo las categóricas que resultaron irrelevantes en la primera parte del TP), para al final del mismo aplicar una reducción de dimensionalidad con el algoritmo PCA. Adicionalmente, probaremos utilizando el normalizador MinMaxScaler en lugar de StandardScaler. Por lo demás, el preprocesamiento se mantiene similar al preprocesamiento 1.

### Aplicamos preprocesamientos 1 y 2 para obtener X_train_1 / X_test_1 y X_train_2 / X_test_2 respectivamente

In [5]:
X_train_1, imputer_entrenado_1, normalizador_1 = preprocesamiento_1(X_train)
X_test_1, imputer_entrenado_1, normalizador_1 = preprocesamiento_1(X_test, imputer_entrenado_1, normalizador_1)

In [6]:
X_train_2, imputer_entrenado_2, normalizador_2 = preprocesamiento_2(X_train)
X_test_2, imputer_entrenado_2, normalizador_2 = preprocesamiento_2(X_test, imputer_entrenado_2, normalizador_2)

### Búsqueda de Hiperparámetros para el modelo que entrenará sobre X_train_1

Buscamos hiperparámetros con RandomGridSearch, pues GridSearch iterativamente es bastante lento. Los parametros a optimizar en un modelo KNN son:
- weights: (pesos) La funcion utilizada para asignarle el peso/importancia a cada punto.
- n_neighbors: (vecinos) el número de vecinos que se observan antes de predecir la clase de una instancia nueva.
- metric: La función utilizada para computar la distancia de los puntos

In [ ]:
params = {'weights': ['distance', 'uniform'], 'n_neighbors':[5, 10, 15],'metric': ['cosine','euclidean']}
hiperparametros = encontrar_hiperparametros_RGSCV(KNeighborsClassifier(), params=params, x_np=X_train_1, y_np=y_train.to_numpy().ravel())

C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
peso_elegido_1 = hiperparametros['weights']
distancia_elegida_1 = hiperparametros['metric']
k_vecinos_elegido_1 = hiperparametros['n_neighbors']
print(f'Mejor peso: {peso_elegido_1}')
print(f'Mejor tipo de distancia: {distancia_elegida_1}')
print(f'Mejor cantidad de K vecinos: {k_vecinos_elegido_1}')

### Entrenando el Modelo 1 sobre X_train_1 con CrossValidation

Procedemos a testear con kfolds, stratificados pues nuestro dataset es desbalanceado. Además, usamos los hiperparámetros encontrados previamente.

In [ ]:
kf = StratifiedKFold(n_splits=5)
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train_1, y_train)):
    knn_clasificacion = KNeighborsClassifier(metric=distancia_elegida_1, n_neighbors=k_vecinos_elegido_1, weights=peso_elegido_1)
    knn_clasificacion.fit(X_train_1[train_index], y_train.iloc[train_index].values.ravel())
    print ('Reporte para el FOLD ' + str(fold_idx))
    print(classification_report(y_train.iloc[test_index], knn_clasificacion.predict(X_train_1[test_index]), target_names=['No llueven hamburguesas al dia siguiente', 'Llueven hamburguesas al dia siguiente']))
    

### Búsqueda de Hiperparámetros para el modelo que entrenará sobre X_train_2

Buscamos hiperparámetros con RandomGridSearch, pues GridSearch iterativamente es bastante lento. Los parametros a optimizar en un modelo KNN son:
- weights: (pesos) La funcion utilizada para asignarle el peso/importancia a cada punto.
- n_neighbors: (vecinos) el número de vecinos que se observan antes de predecir la clase de una instancia nueva.
- metric: La función utilizada para computar la distancia de los puntos

In [ ]:
params = {'weights': ['distance', 'uniform'], 'n_neighbors':[5, 10, 15],'metric': ['cosine','euclidean']}
hiperparametros = encontrar_hiperparametros_RGSCV(KNeighborsClassifier(), params=params, x_np=X_train_2, y_np=y_train.to_numpy().ravel())

In [ ]:
peso_elegido_2 = hiperparametros['weights']
distancia_elegida_2 = hiperparametros['metric']
k_vecinos_elegido_2 = hiperparametros['n_neighbors']
print(f'Mejor peso: {peso_elegido_2}')
print(f'Mejor tipo de distancia: {distancia_elegida_2}')
print(f'Mejor cantidad de K vecinos: {k_vecinos_elegido_2}')

### Entrenando el Modelo 2 sobre X_train_2 con CrossValidation

Procedemos a testear con kfolds, stratificados pues nuestro dataset es desbalanceado. Además, usamos los hiperparámetros encontrados previamente.

In [ ]:
kf = StratifiedKFold(n_splits=5)
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train_2, y_train)):
    knn_clasificacion = KNeighborsClassifier(metric=distancia_elegida_2, n_neighbors=k_vecinos_elegido_2, weights=peso_elegido_2)
    knn_clasificacion.fit(X_train_2[train_index], y_train.iloc[train_index].values.ravel())
    print ('Reporte para el FOLD ' + str(fold_idx))
    print(classification_report(y_train.iloc[test_index], knn_clasificacion.predict(X_train_2[test_index]), target_names=['No llueven hamburguesas al dia siguiente', 'Llueven hamburguesas al dia siguiente']))
    

### Predicción del modelo sobre holdout con el modelo que mejor resultó al hacer CrossValidation: Modelo 2

Podemos ver al comparar ambos procesos de CrossValidation, que en general el modelo que usó el preprocesamiento_2 resultó en mejores métricas provistas por el classification report. En promedio, apreciamos un recall mejor, y en consecuencia también un f1 score mucho mejor para este modelo. Vemos además que no hay manifestaciones de overfit, por lo que pasaremos a utilizar el Modelo 2 para predecir sobre el dataset de holdout.

In [ ]:
knn_holdout = KNeighborsClassifier(metric=distancia_elegida_2, n_neighbors=k_vecinos_elegido_2, weights=peso_elegido_2)
knn_holdout.fit(X_train_2, y_train.to_numpy().ravel())

In [ ]:
mapeo_binario_v = np.vectorize(mapear_target_binario)
y_pred = knn_holdout.predict(X_test_2)
y_pred_binario = mapeo_binario_v(y_pred)
y_pred_proba = knn_holdout.predict_proba(X_test_2)[:, 1]
y_test_binario = y_test['llovieron_hamburguesas_al_dia_siguiente'].map({'si': 1, 'no': 0}).to_numpy()
print(classification_report(y_test['llovieron_hamburguesas_al_dia_siguiente'].to_numpy(), y_pred))

### Curva AUC ROC


In [ ]:
graficar_auc_roc(y_test_binario, y_pred_proba)

Se observa una métrica de auc roc bastante aceptable, en especial si la comparamos con nuestros resultados aplicando T-SNE antes de recibir correcciones al TP. Aún así, sigue siendo inferior a otros modelos presentados en este trabajo práctico, como por ejemplo el árbol de decisión.

# Matriz de confusion

In [ ]:
graficar_matriz_confusion(y_test_binario, y_pred_binario)

Podemos notar que si desprendemos los cálculos de Precision/Recall de la matriz resultante, estos coinciden con los reportados por classification_report(), y por lo tanto la confección de la misma es correcta. Si bien inicialmente vimos que el accuracy de KNN no fue malo, esta matriz nos indica claramente que para otras métricas el modelo no se comporta de manera satisfactoria, obteniendo solo un 0.43 de recall y 0.65 de precision.

## Predicción con el dataset nuevo
A continuación, realizamos la predicción con el modelo KNN sobre el dataset de predicciones nuevo, y la escribimos al archivo 'KNN.csv'

In [ ]:
from funciones_auxiliares import exportar_prediccion_final

df_prediccion_final = traer_dataset_prediccion_final()
ids = df_prediccion_final['id'].to_numpy()

df_prediccion_final, imputer_entrenado_2, normalizador = preprocesamiento_2(df_prediccion_final, imputer_entrenado_2, normalizador_2)

predicciones = knn_holdout.predict(df_prediccion_final)
exportar_prediccion_final(ids, predicciones, 'KNN')